https://golang.org/doc/codewalk/markov/
https://en.wikipedia.org/wiki/Markov_chain
https://en.wikipedia.org/wiki/N-gram#n-gram_models

In [ ]:
import nltk
import random

# create a dict with the ngram model, it receives a list with the samples as string
ngram_dict = {}

def create_ngram_dict(corpus):
    n = 3
    ngrams = nltk.ngrams(corpus, n)
    
    for grams in ngrams:
        dict_key = grams[:-1][0] + " " + grams[:-1][1]
        if dict_key in ngram_dict:
            ngram_dict[dict_key].append(grams[-1])
        else:
            ngram_dict[dict_key] = []
            ngram_dict[dict_key].append(grams[-1])

In [ ]:
def generate(seed, samples = 15):
    output = seed  
    for i in range(samples):
        # When it reaches the last prefix, there is no suffix, so end
        try:
            new_sample = random.choice(ngram_dict[seed])
        except:
            return output
        output += " " + new_sample
        seed = seed.split(" ")[1] + " " + new_sample

    return output

In [ ]:
import pickle

midi_dataset = pickle.load(open("validation_DB12_final_cleaned.p", "rb"))
print("ready!!!")

In [ ]:
corpus = midi_dataset["x"]
corpus_str=map(str, corpus)

In [ ]:
ngram_dict = {}
create_ngram_dict(corpus_str)
print("ready!!!")

In [ ]:
generated_melody = generate("60 62", 100)
print(generated_melody)

In [ ]:
from midiutil.MidiFile import MIDIFile   

# takes a list of integers representing midi notes and creates a .mid
# with a contant time of a quarter note for all the notes and 120 
# as tempo(taken from garcia, 2018)
def sequenceVector2midiMelody(seqVector, file_dir):
    MyMIDI = MIDIFile(1)
    track = 0 
    time = 0
    MyMIDI.addTrackName(track,time,"Sample Track") 
    MyMIDI.addTempo(track,time,120)
    time = 0
    for note in seqVector:
        # MyMIDI.addNote(track,channel,pitch,time,duration,volume)
        MyMIDI.addNote(0,0,note,time,1,100)
        time = time + 1

    binfile = open(file_dir, 'wb') 
    MyMIDI.writeFile(binfile) 
    binfile.close()

In [ ]:
sequenceVector2midiMelody(map(int,generated_melody.split(" ")), 'generated_melody.mid')

In [ ]:
from music21 import midi
import music21
music21.environment.set("musicxmlPath", "/usr/bin/musescore")
music21.environment.set('musescoreDirectPNGPath', '/usr/bin/musescore')
music21.environment.set('midiPath', '/usr/bin/timidity')

mf = midi.MidiFile()
mf.open("generated_melody.mid")
mf.read()
mf.close()
play = midi.translate.midiFileToStream(mf)
play.show("midi")

In [ ]:
play.show("midi")

In [ ]:
from music21.converter.subConverters import ConverterMusicXML
conv_musicxml = ConverterMusicXML()
scorename = 'myScoreName.xml'
filepath = 'C:/path/to/musical_scores/' + scorename
out_filepath = conv_musicxml.write(music21object, 'musicxml', fp=filepath, subformats=['png'])

In [ ]:
print generate("am I", 100)